# Exercise Session 7 - Cross-validation

How should we be choosing the value of $k$ for kNN? If we choose the $k$ that gives us the highest **test** accuracy, we would be **cheating** because we would be tuning our model to its test data. 

In practice, we choose the $k$ that gives us the highest **validation** accuracy, via the cross-validation method. By doing so, we ensure that we select a method which generalizes well to unseen test data.

In this little helper notebook, you will see some pointers to help you implement it.

In [1]:
import numpy as np

from helpers import KNN, mse_fn, macrof1_fn

## 1.  K-Fold Cross Validation 

K-fold is a type of cross validation technique that consists into dividing the dataset into K parts (:= folds) of equal size. Each fold is considered the validation set of the rest of the folds, which are used for training.
It works in the following way:

1 - Split the training data in K folds. Select 1 fold as our validation set and the rest as our training set.

2 - Train our model on the training set and find the accuracy (or other metric value) of the validation set. 

3 - Repeat steps 1&2 K times, each time selecting a different fold of the data for the validation set. 

4 - In the end we will find K different validation accuracies that we average. This will represent the accuracy/performance of our model. (See the image below).

![](cross_validation.png)

This process can be used to choose the best hyper-parameter value for a given model. Indeed, we can repeat it for different values, tracking which one gives the best accuracy/performance. For a kNN example:

a - Repeat steps 1-4 (the whole process) for different $k$ values (hyperparameter for kNN). 

b - Find the $k$ value that gave the highest validation accuracy.

### 1.1 Splitting the data
First, you need to implement `splitting_fn()`, a function that split the data into the specific training and validation folds, e.g. the 4th one on the graph above.

To do the splitting, we first do the following for you:

a - Create an array of indices from 0 to N-1, where N is the number of data points.

b - Shuffle these indices. This is useful to effectively shuffles the data beforehand.

Then, you will have to implement the following steps in your function:

1 - Extract the indices of the *validation* data for the given `fold`. (Corresponding to the blue square in the image above, which should have a size of `fold_size`.)

2 - Extract the indices of the *training* data for the given `fold`. (Corresponding to the gray square in the image above.) **Helper:** you can take a look at `np.setdiff1d`.

3 - Select all the data points corresponding to the training indices, storing them into `train_data`. Do the same for their labels in `train_label`.

4 - Select all the data points corresponding to the validation indices, storing them into `val_data`. Do the same for their labels in `val_label`.

In [45]:
def splitting_fn(data, labels, indices, fold_size, fold):
    """
        Function to split the data into training and validation folds.
        Arguments:
            data (np.array, of shape (N, D)): data (which will be split to training 
                and validation data during cross validation),
            labels  (np.array, of shape (N,)): the labels of the data
            indices: (np.array, of shape (N,)): array of pre shuffled indices (integers ranging from 0 to N)
            fold_size (int): the size of each fold
            fold (int): the index of the current fold.
        Returns:
            train_data, train_label, val_data, val_label (np. arrays): split training and validation sets
    """
    trainslice = np.setdiff1d(np.arange(data.shape[0]), np.arange(fold_size*fold,fold_size*(fold+1)))
    
    train_data = data[indices[trainslice], :]
    train_label = labels[indices[trainslice]]
    val_data = data[indices[fold * fold_size : (fold+1) * fold_size], :]
    val_label = labels[indices[fold * fold_size : (fold+1) * fold_size]]

    return train_data, train_label, val_data, val_label

The following cell is a small test to partially verify the function.

Note: you are encouraged to test it further yourself. What do you expect the outputs to be? Does what you obtain make sense to you? Don't hesitate to print the results.

In [46]:
N = 10
k_fold = 5  # the number of folds
data = np.arange(N).reshape(N,1)
labels = (data >= 4).astype(int).reshape(N)

indices = np.arange(N)  # array on indices from 0 to N-1
np.random.shuffle(indices)  # we shuffle that array
fold_size = N//k_fold

for fold in range(k_fold):
    train_data, train_label, val_data, val_label = splitting_fn(data, labels, indices, fold_size, fold)

    if not all([x not in train_data for x in val_data]):
        print("There seems to be an error in your splitting_fn: train_data and val_data share some data points!")
        break
    

    cat_data = np.concatenate([train_data, val_data], axis=0)
    if not all([x in cat_data for x in data]):
        print("There seems to be an error in your splitting_fn: not all data points are used!")
        break

### 1.2 Performing K-Fold Cross Validation

You need to implement the main function  `cross_validation`.
The aim is to perform such cross-validation over a specific hyper-parameter of the method, given some data and labels.

Again, here are the steps:

1. Repeat for the different parameter values:
    1. Repeat K times, each time selecting a different fold of the data for the validation set:
        1. Split the training data in K folds. Select 1 fold as our validation set and the rest as our training set.
        2. Train our model on the training set and find the accuracy (or other metric value) of the validation set. 
    2. In the end we will find K different validation accuracies that we average. This will represent the accuracy/performance of our model with that parameter.
3. Select the value that gave the highest validation accuracy.


**Note:**
* We talked about accuracy, however it can be generalized to any metric! For example, in the case of regression we are looking at MSE, and we want to minimize it.
    * You can use `metric` and `find_param_ops` below for this.
* We create and shuffle the indices for you, in `indices`.
* To track the metric for each parameter value, you can use `acc_list1`.
* To track the metric for each fold, you can use `acc_list2`.

In [81]:
def cross_validation(method_obj=None, search_arg_name=None, search_arg_vals=[], data=None, labels=None, k_fold=4):
    """
        Function to run cross validation on a specified method, across specified arguments.
        Arguments:
            method_obj (object): A classifier or regressor object, such as KNN. Needs to have
                the functions: set_arguments, fit, predict.
            search_arg_name (str): the argument we are trying to find the optimal value for
                for example, for DummyClassifier, this is "dummy_arg".
            search_arg_vals (list): the different argument values to try, in a list.
                example: for the "DummyClassifier", the search_arg_name is "dummy_arg"
                and the values we try could be [1,2,3]
            data (np.array, of shape (N, D)): data (which will be split to training 
                and validation data during cross validation),
            labels  (np.array, of shape (N,)): the labels of the data
            k_fold (int): number of folds
        Returns:
            best_hyperparam (float): best hyper-parameter value, as found by cross-validation
            best_acc (float): best metric, reached using best_hyperparam
    """
    ## choose the metric and operation to find best params based on the metric depending upon the
    ## kind of task.
    metric = mse_fn if method_obj.task_kind == 'regression' else macrof1_fn
    find_param_ops = np.argmin if method_obj.task_kind == 'regression' else np.argmax

    N = data.shape[0]
    indices = np.arange(N)
    np.random.shuffle(indices)
    fold_size = N//k_fold

    acc_list1 = []
    for arg in search_arg_vals:
        arg_dict = {search_arg_name: arg}
        # this is just a way of giving an argument 
        # (example: for DummyClassifier, this is "dummy_arg":1)
        method_obj.set_arguments(**arg_dict)

        acc_list2 = []
        for fold in range(k_fold):
            train_data, train_label, val_data, val_label = splitting_fn(data, labels, indices, fold_size, fold) 
            method_obj.fit(train_data, train_label)
            predictions = method_obj.predict(val_data)
            acc_list2.append(metric(predictions, val_label))
        
        acc_list1.append(np.mean(acc_list2))
     
    best_index = np.argmax(acc_list1)
    best_hyperparam = search_arg_vals[best_index]
    best_acc = acc_list1[best_index]

    return best_hyperparam, best_acc

We test the function in the following cell.

Note: as a bit of randomness is involved (the shuffling of the indices), you might want to run it a few times to verify if it works.

In [82]:
method_obj = KNN()
search_arg_name = "k"
search_arg_vals = [11, 101, 501]

data = np.load("./hbody_feats_annotated.npy")
labels = np.load("./hbody_labels_annotated.npy")
k_fold = 4

best_hyperparam, best_acc = cross_validation(method_obj, search_arg_name, search_arg_vals, data, labels, k_fold)

if best_hyperparam != 11:
    print("There seems to be an error in your cross_validation: "
          "given this setup, we expect k=11 to be better than 101 and 501!")
    
if best_acc < 0.79:
    print("There seems to be an error in your cross_validation: "
          "given this setup, we expect the macrof1 score to be above 0.79!")